In [71]:

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import plotly.express as px
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re

In [80]:
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")
driver = webdriver.Chrome(executable_path='./chromedriver',options=options)
driver.maximize_window()

url = 'https://brunch.co.kr/search'
driver.get(url)
driver.find_element('id','txt_search').send_keys('행복')
driver.find_element('id','txt_search').send_keys(Keys.ENTER)

last_height = driver.execute_script("return document.body.scrollHeight") # 스크롤 높이 가져옴

count = 0
while count < 300:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    new_height = driver.execute_script("return document.body.scrollHeight") # 스크롤 다운 후 스크롤 높이 다시 가져옴
    if new_height == last_height:
        break
    last_height = new_height
    count+=1


/var/folders/7x/34xl2tz90bqdj51kzjdb7xwh0000gp/T/ipykernel_17320/3960883171.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver',options=options)


In [81]:
def deleteBlank(text) -> str:
    text = text.replace('\n','')
    return re.sub(' +', ' ', text)

In [82]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
mydivs = soup.find_all("div", {"class": "post_title"})

dataframelist = []
for div in mydivs:
    title = div.find("strong", {"class": "tit_subject"}).get_text()
    content = div.find("span", {"class": "article_content"}).get_text()
    date = div.find("span", {"class": "publish_time"}).get_text()
    
    title = deleteBlank(title)
    content = deleteBlank(content)
    
    dataframelist.append([title, content, date])

columns = ["제목", "내용", "날짜"]
df = pd.DataFrame(dataframelist, columns = columns)
with pd.ExcelWriter('brunchResult.xlsx', engine='openpyxl') as writer:
    df.to_excel(writer)